# Answering Business Questions using SQL

In this project we will be working with modified version of Chinook database. The Chinook database contains information about a fictional digital music shop.

Check more about Chinook database here : https://github.com/lerocha/chinook-database

Here we will be using 'chinook.db' (sqlite databse) and write SQL queries to extract the relevant data to answer speific business questions.

Importing the required modules

In [6]:
import sqlite3
import pandas as pd

Here we are importing 'pandas' as we can also use 'pandas' to run queries and it will produce dataframe outputs which will be displayed as tables.
We just need to pass the connection object along with the query to the pd.read_sql() to get the results of the query as a data frame.

First, let us create some helper functions so that we can simply call these functions every time we want to run specific task. (Ex: Running a query, Creating, Inserting or Modifying a table, Listing all tables)

In [7]:
def run_query(query):
    with sqlite3.connect('chinook.db') as conn:
        return pd.read_sql(query, conn)

The above function executes the given query and returns the result as a pandas data frame. We can use this function when our query is returning a table.

Now, let us write another function to run commands that don't return any result tables like 'CREATE', 'INSERT' and 'ALTER' statements.

In [8]:
def run_command(command):
    with sqlite3.connect('chinook.db') as conn:
        conn.isolation_level = None
        conn.execute(command)

In the above function the 'conn.isolation_level = None' tells SQLite to autocommit any changes immediately without waiting until end of transaction block.

Finally, let us also make a function to list all the tables and views in our database.

In [11]:
def show_tables():
    query = '''
            SELECT
                name,
                type
            FROM sqlite_master
            WHERE type IN ("table", "view")
            '''
    return run_query(query)

As we can see in the above function, we made use of existing run_query() function to execute the query to select tables and views from our database.

In [13]:
show_tables()

,name,type
0,album,table
1,artist,table
2,customer,table
3,employee,table
4,genre,table
5,invoice,table
6,invoice_line,table
7,media_type,table
8,playlist,table
9,playlist_track,table


Now let us write a query to see which genre tracks are sold most in the country 'USA'.

In [32]:
query = '''
        WITH absolute_numbers AS 
        (
        SELECT
            g.name AS genre_name,
            SUM(il.quantity) AS no_of_tracks_sold
        FROM (SELECT * FROM customer WHERE country = 'USA') c
        INNER JOIN invoice i ON i.customer_id = c.customer_id
        INNER JOIN invoice_line il ON il.invoice_id = i.invoice_id
        INNER JOIN track t ON t.track_id = il.track_id
        INNER JOIN genre g ON g.genre_id = t.genre_id
        GROUP BY genre_name
        )
        SELECT 
            genre_name, 
            no_of_tracks_sold, 
            CAST(no_of_tracks_sold AS FLOAT)/(SELECT SUM(no_of_tracks_sold) FROM absolute_numbers) AS percentage 
            FROM absolute_numbers
            ORDER BY no_of_tracks_sold DESC
        '''
run_query(query)

,genre_name,no_of_tracks_sold,percentage
0,Rock,561,0.533777
1,Alternative & Punk,130,0.123692
2,Metal,124,0.117983
3,R&B/Soul,53,0.050428
4,Blues,36,0.034253
5,Alternative,35,0.033302
6,Latin,22,0.020932
7,Pop,22,0.020932
8,Hip Hop/Rap,20,0.019029
9,Jazz,14,0.013321


As we can see from the above results, tracks from 'Rock' genre are most sold with more than 50% of total tracks sold in USA.

Now let us write a query to find the total sales done by each of the sales support agent to see if any of them are over performing or under performing.

In [55]:
query = ''' 
        WITH customer_totals AS
        (
        SELECT 
            c.customer_id,
            c.support_rep_id,
            SUM(i.total) as total
        FROM customer c
        INNER JOIN invoice i ON i.customer_id = c.customer_id
        GROUP BY i.customer_id
        )
        SELECT 
            e.first_name || ' ' || e.last_name as emp_name,
            SUM(ct.total) AS total_sales,
            e.hire_date AS hire_date
        FROM employee e
        INNER JOIN customer_totals ct ON ct.support_rep_id = e.employee_id
        WHERE e.title = 'Sales Support Agent'
        GROUP BY e.employee_id
        '''
run_query(query)

,emp_name,total_sales,hire_date
0,Jane Peacock,1731.51,2017-04-01 00:00:00
1,Margaret Park,1584.00,2017-05-03 00:00:00
2,Steve Johnson,1393.92,2017-10-17 00:00:00


In the above result, we have selected the column 'hire_date' to understand the difference in total sales.

We can conclude from the above results that there is a slight difference in amount of 'total_sales' and we can safely attribute it to hiring date as the diffence is very less.

From the 'hire_date' column we see Jane started in January, Margaret started in March and Steve started in October. Thus, we expect a difference in total sales made by them. In conclusion, we can say all of them are performing on same level without anyone outperforming or underperforming than others.

In [89]:
query = '''
        WITH 
            customers_by_country AS
            (
                SELECT 
                    country,
                    COUNT(*) as no_of_customers
                FROM customer
                GROUP BY country
            ),
            sales_by_country AS
            (
                SELECT 
                    c.country,
                    COUNT(*) AS no_of_orders,
                    SUM(i.total) AS total_sales
                FROM customer c
                INNER JOIN invoice i ON i.customer_id = c.customer_id
                GROUP BY c.country
            ),
            unordered_result AS
            (
                SELECT
                    CASE
                        WHEN cc.no_of_customers = 1 THEN 'Others'
                        ELSE cc.country
                    END as country,
                    SUM(no_of_customers) AS no_of_customers,
                    SUM(total_sales) AS total_sales,
                    SUM(total_sales)/SUM(no_of_customers) AS avg_sales_per_customer,
                    SUM(total_sales)/SUM(no_of_orders) AS avg_order_value
                FROM customers_by_country cc
                INNER JOIN sales_by_country sc ON cc.country = sc.country
                GROUP BY 1
                ORDER BY total_sales DESC
            )
        SELECT 
            country, 
            no_of_customers, 
            total_sales,
            avg_sales_per_customer,
            avg_order_value
        FROM 
            (
            SELECT 
                ur.*,
                CASE
                    WHEN ur.country = "Others" THEN 1
                    ELSE 0
                END AS sort
            FROM unordered_result ur
            )
        ORDER BY sort
        '''
run_query(query)

,country,no_of_customers,total_sales,avg_sales_per_customer,avg_order_value
0,USA,13,1040.49,80.037692,7.942672
1,Canada,8,535.59,66.948750,7.047237
2,Brazil,5,427.68,85.536000,7.011148
3,France,5,389.07,77.814000,7.781400
4,Germany,4,334.62,83.655000,8.161463
5,Czech Republic,2,273.24,136.620000,9.108000
6,United Kingdom,3,245.52,81.840000,8.768571
7,Portugal,2,185.13,92.565000,6.383793
8,India,2,183.15,91.575000,8.721429
9,Others,15,1094.94,72.996000,7.448571


From the above results we see the average sales per customer is more in countries 'Czech Republic', 'Portugal', 'India' and no of customers are less in these countries. Thus we can say there is a potential for growth in these countries and the company should invest more in these countries.

Now let us write a query to find how many no of purchases are album purchases and how many no of purchases are not, so that the company can decide on whether it should keep purchasing albums as a whole or to only buy popular tracks from the albums to maximize revenue.

In [115]:
query = '''WITH 
            min_track_id AS
            (
            SELECT 
                invoice_id, MIN(track_id) as track_id 
            FROM invoice_line GROUP BY invoice_id
            )
            SELECT 
                album_purchase,
                COUNT(invoice_id) AS no_of_invoices,
                CAST(COUNT(invoice_id) AS FLOAT) / (SELECT COUNT(*) FROM invoice) AS percentage
            FROM
                (
                SELECT 
                    mti.invoice_id,
                    CASE
                        WHEN 
                            (
                            SELECT il1.track_id 
                            FROM invoice_line il1 
                            WHERE il1.invoice_id = mti.invoice_id

                            EXCEPT

                            SELECT t1.track_id
                            FROM track t1
                            WHERE t1.album_id = (
                                                SELECT t2.album_id
                                                FROM track t2
                                                WHERE mti.track_id = t2.track_id
                                                )
                            ) IS NULL
                        AND
                            (
                            SELECT t1.track_id
                            FROM track t1
                            WHERE t1.album_id = (
                                                SELECT t2.album_id
                                                FROM track t2
                                                WHERE mti.track_id = t2.track_id
                                                )

                            EXCEPT

                            SELECT il1.track_id 
                            FROM invoice_line il1 
                            WHERE il1.invoice_id = mti.invoice_id                        
                            ) IS NULL
                        THEN "yes"
                        ELSE "no"
                        END AS album_purchase
                FROM min_track_id mti
                )
            GROUP BY album_purchase
            '''
run_query(query)

,album_purchase,no_of_invoices,percentage
0,no,500,0.814332
1,yes,114,0.185668


From the above results we see about 20 percent of all the purchases are whole album purchases and rest are purchases of individual tracks.

Thus we can advise to stop purchasing albums as a whole as they only contribute to less than 20 percent of all purchases and suggest to buy popular tracks from different albums to maximize revenue.

Note that customers who are interested in purchasing whole  albums will still be able to get the most popular songs from those albums, so we wont be losing that whole 20% of the revenue. 